# Capstone: Musical Recommender

Kelly Slatery | US-DSI-10

In [1]:
# Imports 
import numpy as np
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import regex as re
import unicodedata

In [2]:
# Set view options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Import Data

In [3]:
# Import list of musicals
names = pd.read_csv('./data/musical_names.csv')
names.shape

(196, 1)

In [4]:
names.head()

,musical
0,Les Misérables
1,The Phantom of the Opera
2,Hamilton
3,West Side Story
4,Wicked


In [5]:
# Make all names lowercase (and fix spelling of "Seussical")
names['musical'] = [name.lower().replace('suessical', 'seussical') for name in names['musical']]
names.head()

,musical
0,les misérables
1,the phantom of the opera
2,hamilton
3,west side story
4,wicked


# Data Collection

In [6]:
# Set up base url
baseurl1 = 'https://www.allmusicals.com/lyrics/'
baseurl2 = '/synopsis.htm'

In [7]:
# Set up empty list for musical synopses from allmusicals.com
synopses = []
missed_musicals = []

# Loop through all musicals in the list
for name in names['musical']:
    
    # Credits (UnicodeDecodeError): @Vincent Marchetti 10.13.2008, https://stackoverflow.com/questions/196345/how-to-check-if-a-string-in-python-is-in-ascii
    # Credits (unicodedata): @Mini Quark 02.05.2009, https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-in-a-python-unicode-string
    try:
        name.encode('utf-8').decode('ascii')
    except UnicodeDecodeError:
        name = unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('utf-8')
    
    # Remove all punctuation (not a character, number, or space)
    name = re.sub(r'[^a-zA-Z0-9 ]+', '', name)
    
    # Copy original name
    og_name = name
    
    # Handle titles starting with "The"
    if name[:3] == 'the':
        
        # Remove whitespace and make lowercase
        name = re.sub('[^a-zA-Z0-9]', '', name)
        
        # Set up list of links to loop over
        possible_names = [name[3:], name[3:] + 'the', name, name + 'the']
        i = 0        
        url = baseurl1 + possible_names[i] + baseurl2
        res = requests.get(url)

        # Check for which version of the musical name returns a result, if any
        while res.status_code != 200:
            i += 1
            try:
                url = baseurl1 + possible_names[i] + baseurl2
            except IndexError:
                url = ''
                break
            res = requests.get(url)
    
    # Handle titles starting with "A"
    elif name.strip().split()[0] == 'a':
        
        # Remove whitespace and make lowercase
        name = re.sub('[^a-zA-Z0-9]', '', name)
        
        # Set up list of links to loop over
        possible_names = [name[1:], name[1:] + 'a', name, name + 'a']
        i = 0        
        url = baseurl1 + possible_names[i] + baseurl2
        res = requests.get(url)
        
        # Check for which version of the musical name returns a result, if any
        while res.status_code != 200:
            i += 1
            try:
                url = baseurl1 + possible_names[i] + baseurl2
            except IndexError:
                url = ''
                break
            res = requests.get(url)
    
    # Remove "the musical" from names
    elif 'the musical' in name:
        name = name.replace('the musical', '')
        name = re.sub('[^a-zA-Z0-9]', '', name)
        url = baseurl1 + name + baseurl2
        res = requests.get(url)

    # Access webpage for all musicals not containing "The"
    else: 
        name = re.sub('[^a-zA-Z0-9]', '', name)
        url = baseurl1 + name + baseurl2
        res = requests.get(url)
            
    # Try a few more variants of the musical name
    if res.status_code != 200:
        new_name = og_name.split()[0]
        url = baseurl1 + new_name + baseurl2
        res = requests.get(url)
        if res.status_code != 200:
            new_name = ''.join(og_name.split()[:-1])
            url = baseurl1 + new_name + baseurl2
            res = requests.get(url)
            if res.status_code != 200:
                new_name = name + 'themusical'
                url = baseurl1 + new_name + baseurl2
                res = requests.get(url)
                if res.status_code != 200:
                    new_name = name + 'musical'
                    url = baseurl1 + name + baseurl2
                    res = requests.get(url)
                    if res.status_code != 200:
                        new_name = og_name.replace('and', '')
                        new_name = re.sub('[^a-zA-Z0-9]', '', new_name)
                        url = baseurl1 + new_name + baseurl2
                        res = requests.get(url)
                        if res.status_code != 200:
                            synopses.append('')
                            missed_musicals.append(og_name)
                            print(f'NOT LISTED: "{og_name}". Try another source like Wikipedia.')
                            continue

    # For all plausible muical names searches, get the synopsis
    soup = BeautifulSoup(res.content)
    synopsis_html = soup.find('div', {'id': 'page'}).text.replace('\r', '')
    synopsis_list = [words.strip() for words in synopsis_html.split('\n')[2:-2]]
    synopsis = ' '.join(synopsis_list).strip()
    synopses.append(synopsis)
    print(f'Synopsis for "{name}" added to list. Moving on to the next musical...')

Synopsis for "lesmiserables" added to list. Moving on to the next musical...
Synopsis for "thephantomoftheopera" added to list. Moving on to the next musical...
Synopsis for "hamilton" added to list. Moving on to the next musical...
Synopsis for "westsidestory" added to list. Moving on to the next musical...
Synopsis for "wicked" added to list. Moving on to the next musical...
Synopsis for "chicago" added to list. Moving on to the next musical...
Synopsis for "rent" added to list. Moving on to the next musical...
Synopsis for "thelionking" added to list. Moving on to the next musical...
Synopsis for "thebookofmormon" added to list. Moving on to the next musical...
Synopsis for "sweeneytoddthedemonbarberoffleetstreet" added to list. Moving on to the next musical...
Synopsis for "thesoundofmusic" added to list. Moving on to the next musical...
Synopsis for "fiddlerontheroof" added to list. Moving on to the next musical...
Synopsis for "intothewoods" added to list. Moving on to the next m

Synopsis for "beautifulthecarolekingmusical" added to list. Moving on to the next musical...
Synopsis for "brigadoon" added to list. Moving on to the next musical...
Synopsis for "onthetown" added to list. Moving on to the next musical...
Synopsis for "thewhostommy" added to list. Moving on to the next musical...
Synopsis for "bemorechill" added to list. Moving on to the next musical...
Synopsis for "thewiz" added to list. Moving on to the next musical...
Synopsis for "funhome" added to list. Moving on to the next musical...
Synopsis for "catchmeifyoucan" added to list. Moving on to the next musical...
Synopsis for "sweetcharity" added to list. Moving on to the next musical...
Synopsis for "mame" added to list. Moving on to the next musical...
Synopsis for "sevenbridesforsevenbrothers" added to list. Moving on to the next musical...
Synopsis for "sisteract" added to list. Moving on to the next musical...
Synopsis for "shelovesme" added to list. Moving on to the next musical...
Synopsis

In [8]:
# Check that length of synopses is 196
print(len(synopses))

# Look at which musicals are missing synopses
print(missed_musicals)

196
['grease', 'beauty and the beast', 'cinderella', 'porgy and bess', 'once', 'pajama game', 'bonnie  clyde', 'beetlejuice', 'tuck everlasting', 'dr seuss how the grinch stole christmas', 'oh calcutta', 'dogfight', 'twisted', 'shenandoah', 'coco', 'cyrano', 'ballroom', 'the magic show', 'dancin']


In [9]:
# How many musicals were missed?
len(missed_musicals)

19

# Export Data

In [10]:
# Add synopses to dataframe
names['synopsis'] = synopses
names.head(2)

musical  \
0            les misérables   
1  the phantom of the opera   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [11]:
# Export new dataframe
names.to_csv('./data/musical_synopses.csv', index=False)